GAME (detecting cis-regulatory elements using a genetic algorithm. Wei Z1, Jensen ST) in Julia
Team member: Mayank, Mohammad, Ian



In [26]:
# We annotate the functions we use with its use, parameters, output, and time complexity (if we think it is relevant)
# We skip some of the above information for functions that we feel are self explanatory with the intention of keeping the code clean
# There is a total of 4 fasta files for the easy and hard challenges as described in our report and 1 fasta file for CRP data

In [ ]:
# Data related notations
# t -> Number of DNA sequences
# l -> Length of a DNA sequence
# w -> width of motif

# GAME related notations
# N -> Size of population
# max generation steps -> maximum number of steps for which the genetic component of GAME runs
# r -> Mutation rate of the genetic component of GAME. The probability with which each start site
#      is mutated in a member of the population

In [7]:
import Statistics
import Random
import StatsBase

Functions for simulating biological data

In [8]:
# Algorithm: Generate random set of DNA sequences of given size and quantity
# Parameters: Number of DNA sequences t, Length of sequence l
# Output: t randomly generated DNA sequences
# Time Complexity: O(t*l) 
function generate_random_sequences(number_of_sequences, sequence_length)
    generated_sequences = []
    for i in 1:number_of_sequences
        generated_sequence = Random.randstring("ACGT", sequence_length)
        push!(generated_sequences, generated_sequence)
    end
    return generated_sequences
end

# Algorithm: Generate a random population of potential motif profiles of a given size for a given set of DNA sequences
# Parameters: Size of population N, Number of DNA sequences t, Length of sequence l, Lenght of motif w
# Output: N randomly generated motif profiles of width w for t DNA sequences of length l
# Time Complexity: O(N*t)
function generate_random_initial_population(population_size, number_of_sequences, sequence_length, motif_width)
    generated_population = []
    for i in 1:population_size
        motif_start_point_range = 1:sequence_length - motif_width + 1
        member_size = number_of_sequences
        generated_member = rand(motif_start_point_range, member_size) # Generate a member of the population of size t
        push!(generated_population, generated_member)
    end
    return generated_population
end

# Function: Generate a random motif string of a given size
# Parameters: Lenght of motif w
# Output: Randomly generated motif string of width w
# Time Complexity: O(w)
function generate_random_motif(motif_length)
   return Random.randstring("ACGT", motif_length)
end

generate_random_motif (generic function with 1 method)

Functions for Mutation, Crossover, Selection

In [9]:
# Helper functions for mutation step in the genetic algorithm


# Count how many members of the population were mutated
function count_mutated_members(initial_population, mutated_population)
    mutated_members = 0
    for (i, j) in zip(initial_population, mutated_population) 
        if any(i .!= j)
            mutated_members += 1
        end
    end
    return mutated_members
end


# Count how many point mutations occurred across all members of the population
function count_mutations(initial_population, mutated_population)
    mutations = 0
    for (i, j) in zip(initial_population, mutated_population)
        mask = (i .!= j)
        mutations += StatsBase.countmap(mask)[true]
    end
    return mutations
end

function to_mutate(mutation_probability)
    number = rand(1)[1]
    if number  <= mutation_probability
        return true
    else
        return false
    end
end

# Algorithm: Perform mutations over a population of candidate motifs with a given mutation probability assuming uniform sequence legnths
# Parameters: Population of motif profiles, mutation probability, length of sequence, expected width of motif
# Output: Mutated population
# Time complexity : O(N*t) 
function mutation(initial_population, mutation_probability, sequence_length, motif_width)    
    mutated_population = [] # mutated_population
    for i in 1:length(initial_population)
        current_candidate = deepcopy(initial_population[i])
        for j in 1:length(current_candidate)
            if to_mutate(mutation_probability)
                current_candidate[j] = rand(1:sequence_length-motif_width+1)
            end
        end
        push!(mutated_population, current_candidate)
    end
    return mutated_population
end

mutation (generic function with 1 method)

In [10]:
# Algorithm: Perform crossover step
# Parameters: Population of motif profiles
# Output: Post-crossover population
# Time Complexity: O(N*t)
function crossover(initial_population)    
    shuffled_initial_population = initial_population[Random.randperm(length(initial_population))] # Shuffles/randomizes initial population
    crossover_population = [] # Will hold children produced by crossover (N/2 pairs)
    
    for i in 1:2:length(shuffled_initial_population) # index by 2
        if(i+1 <= length(shuffled_initial_population))
            A = shuffled_initial_population[i]
            B = shuffled_initial_population[i+1]
            crossOverPoint = rand(1:length(A)) # cross over point; we assume A and B are of the same length (m)
        
            A_prime = []
            B_prime = []
            for j in 1:crossOverPoint
                push!(A_prime, A[j])
                push!(B_prime, B[j])
            end
            for j in crossOverPoint+1:length(A)
                push!(A_prime, B[j])
                push!(B_prime, A[j])
            end

            push!(crossover_population, A_prime)
            push!(crossover_population, B_prime)
        end
    end
    
    return vcat(shuffled_initial_population, crossover_population)
end

crossover (generic function with 1 method)

In [11]:
# Algorithm: Perform mutations over a population of candidate motifs with a given mutation probability assuming uniform sequence legnths
# Parameters: Population of motif profiles, number of sequences, expected width of motif, estimated background frequency
# Output: Post-selection population
# Time Complexity: O(N*t*w)
function tournamentSelection(population,tSize,seq,motifWidth, theta0k)
    ### Population is group into tournaments of size tSize.
    ### Output Winners of tournaments
    
    tournamentPopulation = population[Random.randperm(size(population)[1]),:]
    Winners = []
    
    for j in 1:tSize:(size(tournamentPopulation)[1]-tSize+1)


        bracket = tournamentPopulation[j:j+tSize-1,:]
        
        topScore = -1*typemax(Int64) ### Worst possible score is large number
        bracketWinner = -1
        for b in 1:size(bracket)[1]
            # CurrentAlignment = [seq[idx][bracket[b][idx]:bracket[b][idx]+motifWidth-1] for idx in 1:length(seq)]
            if BAYESIAN_SCORING
                currentScore = Score(bracket[b], seq, motifWidth, theta0k)
            else
                motif_string = GenerateMotifStrings(bracket[b], motifWidth, seq)
                currentScore = ConsensusScore(motif_string)
            end
            if(topScore < currentScore )
                topScore = currentScore
                bracketWinner = bracket[b]
            end
        end
       push!(Winners,bracketWinner)
    end
    return Winners
end

tournamentSelection (generic function with 1 method)

Data type changing function

In [12]:
# Algorithm: Generate motifs as a list of DNA sub-strings from a list of starting indices
# Parameters: Motif profile as list of start sites, Width of motif, t DNA sequences
# Output: Motif profile as strings
# Time Complexity : O(t*w)
function GenerateMotifStrings(motifs, motifWidth, DNA_sequences)
  motif_strings = []
  for i in 1:length(motifs)
      start = motifs[i]
      push!(motif_strings, DNA_sequences[i][start:start+motifWidth-1]) 
  end
  return motif_strings
end

GenerateMotifStrings (generic function with 1 method)

Adjust and Shift operations

In [13]:
# Algorithm: Adjust the current optimal motif one start site at a time as described in the GAME paper and the report
# Parameters: Optimal motif profile, DNA sequences, Lenght of a sequence, Number of sequences
# Output: Adjusted motif
# Time Complexity: O(l* t^3 * w^2) (WORST case [avg case is likely better]: score of best profile changes only by 1 every t iterations)
function Adjust(motif, DNA_sequences, motif_width, sequence_length, number_of_sequences)
    i = 0
    motif_adjust = deepcopy(motif)
    score_changes = true
    best_score_for_i = -Inf
    last_score_at_m = -Inf
    first_time = true
    best_profile_for_i = motif_adjust
    k = 1
    while score_changes
        i += 1
        
        if i > number_of_sequences
            i = 1
        end
        
        # Find best motif start value j value for i'th sequence
        for j in 1:sequence_length - motif_width + 1   #takes l*t*w (length of sequence * number of seq * width of motif)
            k += 1
            motif_adjust = vcat(best_profile_for_i[1:i-1], [j], best_profile_for_i[i+1:number_of_sequences])
            motif_strings = GenerateMotifStrings(motif_adjust, motif_width, DNA_sequences)
            score = ConsensusScore(motif_strings)
            if score > best_score_for_i
                 best_profile_for_i = deepcopy(motif_adjust)
                 best_score_for_i = score
            end
        end
        if i == number_of_sequences
            if best_score_for_i > last_score_at_m && !first_time
                last_score_at_m = best_score_for_i
                score_changes = true
            elseif best_score_for_i > last_score_at_m && first_time
                last_score_at_m = best_score_for_i
                first_time = false
            else
                score_changes = false
            end
        end
    end
    return best_profile_for_i
end

Adjust (generic function with 1 method)

In [14]:
# Algorithm: Shift the current optimal motif locally as described in the GAME paper and the report
# Parameters: Optimal motif profile, expected motif width, DNA sequences
# Time Complexity: O(t*w^2) (number of seq * (motif width)^2)
function Shift(motif,motif_width,DNA_sequences)
    shiftedStarts = [[clamp(motif[i]+k,1,length(DNA_sequences[i])-motif_width+1) for i in 1:length(motif)] for k in (-1*motif_width):(motif_width)]
    # k′ = argmax Score(a1+k,………,am+k),−w≤k≤w 
    motif_sequences = map(motif -> GenerateMotifStrings(motif, motif_width, DNA_sequences), shiftedStarts)
    currentScores = map(ConsensusScore, motif_sequences)
    # A←A(a1+k′………,am+k′) Return best shifted 
    return(shiftedStarts[argmax(currentScores)])
end

Shift (generic function with 1 method)

Scoring functions

In [15]:
function EstimateBackground(seq)
  profile = Dict('A' => 0.0, 'C' => 0.0, 'T' => 0.0, 'G' => 0.0)
  totalNumberOfBases = 0.0 #Total number of bases in the sequence
  
  for s in 1:length(seq)
    for b in 1:length(seq[s])
      profile[seq[s][b]] += 1.0
      totalNumberOfBases += 1.0
    end
  end
  
  bases = ['A', 'C', 'G', 'T']
  for i in 1:length(bases)
    profile[bases[i]] /= totalNumberOfBases #Converts counts to frequencies
  end
  
  return(profile)
end

function EstimateBackground(seq, motifs, motifLength)
    background_frequency = Dict('A' => 0.0, 'C' => 0.0, 'T' => 0.0, 'G' => 0.0)
    for sequence_no in 1:length(seq)
       motif_excluded_range = [1:sequence_no - 1 ; sequence_no + motifLength: length(seq[sequence_no])]
       for column_no in motif_excluded_range
           background_frequency[seq[sequence_no][column_no]] += 1
       end
    end
  
  return background_frequency
end

EstimateBackground (generic function with 2 methods)

In [16]:
# Algorithm: Score amotif profile according to consensus scoring as defined in our textbook (Pevzner)
# Parameters: Motif profile
# Output: Integer score
# Time Complexity: O(t*w)
function ConsensusScore(motifs) # Uses Laplace's Rule of Succession
    totalScore = 0
    
    for i in 1:length(motifs[1]) #Goes through every character in a motif
        basesForCurrentCharacter = [] #Holds the bases found in character position i for each motif
        for j in 1:length(motifs) #Goes through every motif, and adds base at position i for current motif to array
            push!(basesForCurrentCharacter, motifs[j][i])
        end
        
        counts = [0, 0, 0, 0]   # [A,C,G,T]
        for j in 1:length(basesForCurrentCharacter) #Increases base count by 1 for corresponding base in current motif character
            if(basesForCurrentCharacter[j] == 'A')
                counts[1] = counts[1] + 1
            elseif(basesForCurrentCharacter[j] == 'C')
                counts[2] = counts[2] + 1
            elseif(basesForCurrentCharacter[j] == 'G')
                counts[3] = counts[3] + 1
            elseif(basesForCurrentCharacter[j] == 'T')
                counts[4] = counts[4] + 1
            end
        end
        consensusCount = maximum(counts) #finds consensus base
        totalScore = totalScore + consensusCount
    end
    
    return totalScore  #Returns total score
end

ConsensusScore (generic function with 1 method)

In [17]:
# Algorithm: Generate consensus string from a motif profile of strings
# Parameters: Motif profile as strings
# Output: Consensus string
# Time Complexity:  O(t*w) 
function ConsensusString(motifs) # Uses Laplace's Rule of Succession
    finalString = ""
    
    for i in 1:length(motifs[1]) #Goes through every character in a motif
        basesForCurrentCharacter = [] #Holds the bases found in character position i for each motif
        for j in 1:length(motifs) #Goes through every motif, and adds base at position i for current motif to array
            push!(basesForCurrentCharacter, motifs[j][i])
        end
        
        counts = [0, 0, 0, 0]   # [A,C,G,T]
        for j in 1:length(basesForCurrentCharacter) #Increases base count by 1 for corresponding base in current motif character
            if(basesForCurrentCharacter[j] == 'A')
                counts[1] = counts[1] + 1
            elseif(basesForCurrentCharacter[j] == 'C')
                counts[2] = counts[2] + 1
            elseif(basesForCurrentCharacter[j] == 'G')
                counts[3] = counts[3] + 1
            elseif(basesForCurrentCharacter[j] == 'T')
                counts[4] = counts[4] + 1
            end
        end
        characterIndex = 1
        maxOccurrence = 0
        for j in 1:length(counts)
            if(counts[j] > maxOccurrence)
                characterIndex = j
                maxOccurrence = counts[j]
            end
        end
        baseCharacter = ""
        if(characterIndex == 1)
            baseCharacter = "A"
        elseif(characterIndex == 2)
            baseCharacter = "C"
        elseif(characterIndex == 3)
            baseCharacter = "G"
        elseif(characterIndex == 4)
            baseCharacter = "T"
        end
        
        finalString = finalString * baseCharacter
    end
    
    return finalString  #Returns consensusString
end

ConsensusString (generic function with 1 method)

In [18]:
# Algorithm: Bayesian scoring function from GAME paper
# Parametes: Motif profile, DNA sequences, expected width of motif, Estimated background frequency
# Output: Real number score
# Time Complexity: O(t*w)
function Score(A, seq, motifLength, theta0k)
    theta0k = EstimateBackground(seq, A, motifLength)
    L = sum(length.(seq).-(motifLength).+1)
    pHat = length(A) / L
    alignment = [seq[idx][A[idx]:A[idx]+motifLength-1] for idx in 1:length(seq)]
    profile = Dict('A' => [0.0 for i in 1:motifLength], 'C' => [0.0 for i in 1:motifLength], 'T' => [0.0 for i in 1:motifLength], 'G' => [0.0 for i in 1:motifLength])
    
    for i in 1:length(alignment) #Goes through number of k_mers 
        for j in 1:length(alignment[i]) #Goes through length of k_mer
            profile[uppercase(alignment[i][j])][j] += 1 #Increase count of base (A, C, G, or T) for current base in k_mer
        end
    end
  
    for i in 1:length(alignment) # Goes through width of k-mer, applies Laplace's rule
        profile['A'][i] = profile['A'][i] + 1
        profile['C'][i] = profile['C'][i] + 1
        profile['G'][i] = profile['G'][i] + 1
        profile['T'][i] = profile['T'][i] + 1
    end
    for i in 1:length(length(alignment)) # Goes through width of k-mer, divides by (2*t) for Laplace's rule of succession
        profile['A'][i] = profile['A'][i]/(2*length(alignment))
        profile['C'][i] = profile['C'][i]/(2*length(alignment))
        profile['G'][i] = profile['G'][i]/(2*length(alignment))
        profile['T'][i] = profile['T'][i]/(2*length(alignment))
    end
  
    bases = ['A', 'C', 'G', 'T']
    productOverMotifFreq = 1
    for i in 1:length(bases)
        for j in 1:length(motifLength)
          productOverMotifFreq *= ( profile[bases[i]][j] * (log(profile[bases[i]][j]) - log(theta0k[bases[i]])) )
        end
    end
  return(length(A) * (log(pHat/(1.0-pHat)) - 1 + productOverMotifFreq))
end

Score (generic function with 1 method)

In [19]:
# Algorithm: Find the precision for GAME on a particular dataset
function precisionGetter(gameResults, injectedMotif, motifWidth, sequences)
    bestPosition = 1
    bestScore = 0
    for i in 1:length(gameResults)
        if(results[i][5] > bestScore)
            bestScore = gameResults[i][5]
            bestPosition = i
        end
    end
    
    startPositions = gameResults[bestPosition][4]
    profile = GenerateMotifStrings(startPositions, motifWidth, sequences)
    consensusString = ConsensusString(profile)
    
    precision = 0.0
    for i in 1:length(consensusString)
        if(consensusString[i] == injectedMotif[i])
            precision += 1.0
        end
    end
    precision = precision/(motifWidth*1.0)

    return precision
end

precisionGetter (generic function with 1 method)

Helper functions for Experimentation

In [20]:
function injectMotif(singleSequence, motif, injectionSite, mutationRate=0.0) #Use for testing since we will know where motif is in this case
  motifWidth = length(motif)
  for i in 1:motifWidth
        nucleotides = ['A', 'C', 'G', 'T']
        if(rand(1)[1] <= mutationRate)
            nucleotides_2 = []    #Holds all nucleotides that ARE NOT current nucleotide in motif
            for j in 1:length(nucleotides)
                if(nucleotides[j] != motif[i])
                    push!(nucleotides_2, nucleotides[j])
                end
            end
            
            mutatedNucleotide = rand(nucleotides_2) #Randomly select nucleotide from array
            
            motif = motif[1:i-1] * mutatedNucleotide * motif[i+1:motifWidth] # Modify mutated nucleotide
        end
  end
    
    seqWithInjectedMotif = ""
  if (injectionSite+motifWidth-1 <= length(singleSequence))  
      seqWithInjectedMotif = singleSequence[1:injectionSite-1] * motif * singleSequence[injectionSite:length(singleSequence)] ### not allowed to edit strings like this
  else
    @warn "Bad motif Injection site"
    @warn injectionSite+motifWidth-1, injectionSite, motifWidth, length(singleSequence)
  end
    
  return seqWithInjectedMotif
end

injectMotif (generic function with 2 methods)

GAME algorithm implementation

In [21]:
# GAME Algorithm
# Parameters: DNA sequences, mutation probability, max generation steps, population size, expected width of motif
# Output: Max score of final population from the genetic component, Mean, Median, Optimal motif, Optimal motif score
# Time Complexity: O(l*w^2*t^3)
function GAME(initial_sequences=[],mutation_probability=0.001,max_gen=3000, population_size=50, motif_width=10)
      # Initialize random sequences if not supplied as a parameter 
      if initial_sequences == []
          initial_random_sequences = generate_random_sequences(number_of_sequences, sequence_length) 
      else
          initial_random_sequences = initial_sequences
      end    
      seq = initial_random_sequences
      sequence_length = length(seq[1])
      number_of_sequences = length(seq)
    
      # Initialize a random popualtion
      initial_random_population =  generate_random_initial_population(population_size, number_of_sequences, sequence_length, motif_width) 

      # Parameters for convergence and selection size
      consecutiveSameAopt = 1
      tournamentSize= 2
    
      # Lists are mutable objects in Julia and a copy must be made in order to preserve original data
      initial_population = deepcopy(initial_random_population)

      # Calculate a-priori background frequencies
      theta0k = EstimateBackground(initial_random_sequences)
     
      # Variable initializations
      i = 0
      previousAopt = 0
      Aopt = -1
      Aopt_motif = []
      maxScores = []
      meanScores = []
      medianScores = []
      adjustShiftScoreArray = []

      # (I) Genetic Part
      # Iterate evolution until maximum generation is reached, or until optimal motif, Aopt fails to change much for 50 iterations (As defined in GAME paper)
      while((i < max_gen) & (consecutiveSameAopt < 50)) #m (max number of generations)
        mutated_population =  mutation(initial_population, mutation_probability, sequence_length, motif_width) 
        crossed_over_population = crossover(mutated_population)  #N*t (pop size * number of seq)
        selected_population =  tournamentSelection(crossed_over_population, tournamentSize, initial_random_sequences, motif_width,theta0k) #N*t*w (pop size * number of seq * width of motif)
        initial_population = selected_population
        
        # Choose scoring function and score the current generation
        if BAYESIAN_SCORING
            currentScores = map(motif -> Score(motif, initial_random_sequences, motif_width, theta0k), selected_population)
        else
            selected_population_strings = map(motif -> GenerateMotifStrings(motif, motif_width, initial_random_sequences), selected_population) #N*t*w (pop size * number of seq * width of motif)
            currentScores = map(ConsensusScore, selected_population_strings) #t*w (number of seq * width of motif)
        end
        
        # Compute and store common stasitical information
        push!(maxScores, maximum(currentScores))
        push!(meanScores, Statistics.mean(currentScores))
        push!(medianScores, Statistics.median(currentScores))
        
        # Pick best motif profile from the current generation
        Aopt = argmax(currentScores)

        i+=1
        
        # Logic for conevrgence
        if(currentScores[Aopt] == previousAopt)
          consecutiveSameAopt+=1
        else
          consecutiveSameAopt=1
        end
        previousAopt = currentScores[Aopt]
        Aopt_motif = selected_population[Aopt]
      end
     
      # (II) Shift + Adjust Cycles to escape local optima
      adjusted = -1
      while(Aopt_motif != adjusted) 
            Aopt_motif = Adjust(Aopt_motif, seq, motif_width, sequence_length, number_of_sequences) 
            Aopt_motif = Shift(Aopt_motif,motif_width,seq)
            adjusted = Aopt_motif
      end
    
      # Choose a scoring function and score the optimal motif
      if BAYESIAN_SCORING
        Aopt_score = Score(Aopt_motif, initial_random_sequences, motif_width, theta0k)
      else
        Aopt_string = GenerateMotifStrings(Aopt_motif, motif_width, initial_random_sequences)  
        Aopt_score =  ConsensusScore(Aopt_string) 
      end
       
    
      return maxScores, medianScores, meanScores, Aopt_motif, Aopt_score
end



GAME (generic function with 6 methods)

Tests

Test for CRP data as described in Final report

In [22]:
# Read the seqeunces from the CRP Fasta file
sequences = []
open("CRP_TF_seq.fasta") do file
    i = 0
    for ln in eachline(file)
        i += 1 
        if i % 2 == 0
            println( "Sequence ", (i // 2).num, ": ", "$(ln)")
            push!(sequences, strip(ln))
        end
    end
end


Sequence 1: TAATGTTTGTGCTGGTTTTTGTGGCATCGGGCGAGAATAGCGCGTGGTGTGAAAGACTGTTTTTTTGATCGTTTTCACAAAAATGGAAGTCCACAGTCTTGAC
Sequence 2: GACAAAAACGCGTAACAAAAGTGTCTATAATCACGGCAGAAAAGTCCACATTGATTATTTGCACGGCGTCACACTTTGGCTATGCCATAGCATTTTTATCCAT
Sequence 3: ACAAATCCCAATAACTTAATATGGGATTTGTTATATATAACTTTATAAATTCCTAAAATTACACAAAGTTAATAACTGTGAGCATGGTCATATTTTTATCAAT
Sequence 4: CACAAAGCGAAAGCTATGCTAAAACAGTCAGGATGCTACAGTAATACATTGATGTACTGCATGTATGCAAAGGACGTCACATTACCGTGCAGTACAGTTGATA
Sequence 5: ACGGTGCTACACTTGTATGTAGCGCATCTTTCTTTACGGTCAATCAGCATGGTGTTAAATTGATCACGTTTTAGACCATTTTTTCGTCGTGAAACTAAAAAAA
Sequence 6: AGTGAATTATTTGAACCAGATCGCATTACAGTGATGCAAACTTGTAAGTAGATTTCCTTAATTGTGATGTGTATCGAAGTGTGTTGCGGAGTAGATGTTAGAA
Sequence 7: GCGCATAAAAAACGGCTAAATTCTTGTGTAAACGATCCACTAATTTATTCCATGTCACACTTTTCGCATCTTTGTTATGCTATGGTTATTTCATACCATAAGC
Sequence 8: GCTCCGGCGGGGTTTTTTGTTATCTGCAATTCAGTACAAAACGTGATCAACCCCTCAATTTTCCCTTTGCTGAAAAATTTTCCATTGTCTCCCCTGTAAAGCT
Sequence 9: AACGCAATTAATGTGAGTTAGCTCACTCATTAGGCACCCCAGGCTTTACACTTTATGCTT

In [23]:
# Use Consensus Scoring
BAYESIAN_SCORING = false
# Run GAME with the default parameters as given in the paper and expected motif width
@time GAME(sequences,0.001,3000, 500, 16)

  5.968905 seconds (38.00 M allocations: 3.185 GiB, 7.41% gc time)


(Any[134, 134, 136, 136, 136, 136, 136, 138, 138, 139  …  170, 170, 170, 170, 170, 170, 170, 170, 170, 170], Any[117.0, 119.0, 121.0, 122.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0  …  170.0, 170.0, 170.0, 170.0, 170.0, 170.0, 170.0, 170.0, 170.0, 170.0], Any[116.984, 119.13, 121.04, 122.64, 124.078, 125.382, 126.458, 127.398, 128.186, 129.246  …  169.996, 170.0, 169.994, 170.0, 170.0, 169.988, 170.0, 170.0, 169.992, 170.0], [2, 40, 39, 38, 49, 38, 10, 74, 72, 62, 57, 32, 64, 7, 5, 19, 83, 7], 186)

In [24]:
q1 = GenerateMotifStrings([2, 40, 39, 38, 49, 38, 10, 74, 72, 62, 57, 32, 64, 7, 5, 19, 83, 7], 16, sequences)
 ConsensusString(q1)

"AAAATTTTAAATTGTT"

In [1]:
# Algorithm: generate challenges described in the report
function generate_challenges(number_of_sequences, sequence_length, motif_length, m, mode)
    sequences = generate_random_sequences(number_of_sequences, sequence_length)
    motif_to_inject = generate_random_motif(motif_length)
    injected_sequences = []
    inject_positions = ""
    println(sequences)
    for sequence in sequences
        position = rand(2:sequence_length-motif_length+1)
        inject_positions *=  string(position) * ","
        push!(injected_sequences, injectMotif(sequence, motif_to_inject, position, m))
    end
    return injected_sequences, inject_positions
end

generate_challenges (generic function with 1 method)

In [2]:
# Challenges generated separately using above function

Tests for running time and precision

Easy Challenge with 100% Motif Conservation

In [33]:
easy_sequences = []
open("Final Generated Fasta Files/Easy GGATAAAAATAAAAGCGGAT-35,50,17,2,41,46,49,25,14,6,22,39,5,9,38,8,6,17,8,5,.fasta") do file
    i = 0
    for ln in eachline(file)
        i += 1 
        if i % 2 == 0
            push!(easy_sequences, strip(ln))
        end
    end
end
BAYESIAN_SCORING = false

results = []
scores = []
times = []
for i in 1:10
    (result, t) = @timed GAME(easy_sequences,0.001,3000, 500, 20)
    push!(times, t)
    push!(results, result)
    push!(scores, result[5])
end
precision = precisionGetter(results, "CGCTCTCCAGCTCGTAGCTC", 20, easy_noisy_sequences)
println("Precision:\t", precision)
println("Average Score:\t", Statistics.mean(scores))
scores

Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[35, 50, 17, 2, 41, 46, 49, 25, 14, 6, 22, 19, 5, 9, 38, 8, 6, 17, 8, 5])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[35, 50, 17, 2, 41, 40, 49, 25, 14, 31, 22, 39, 5, 9, 38, 8, 6, 17, 9, 5])
└ @ Main In[14]:64


Aopt1


...

Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[34, 49, 16, 1, 40, 45, 45, 24, 13, 5, 21, 38, 4, 8, 37, 7, 3, 16, 7, 4])
└ @ Main In[14]:64


Aopt1
397.8


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[36, 51, 18, 3, 42, 47, 50, 26, 15, 6, 23, 40, 6, 9, 39, 9, 7, 18, 9, 6])
└ @ Main In[14]:64


10-element Array{Any,1}:
 400
 400
 400
 390
 400
 400
 400
 388
 400
 400

In [34]:
times

10-element Array{Any,1}:
 10.678517272
  6.833263487
  4.029564223
  7.632428968
  6.489044327
  6.260295558
  4.16709661 
  8.116985373
  5.104873432
  5.320168108

In [35]:
Statistics.mean(times)

6.4632237358

Easy Challenge with 70% Motif Conservation

In [36]:
easy_noisy_sequences = []
open("Final Generated Fasta Files/Easy Noisy CGCTCTCCAGCTCGTAGCTC-8,9,7,44,3,4,16,42,46,45,20,23,36,38,7,3,23,2,33,41,.fasta") do file
    i = 0
    for ln in eachline(file)
        i += 1 
        if i % 2 == 0
            push!(easy_noisy_sequences, strip(ln))
        end
    end
end
BAYESIAN_SCORING = false

results = []
scores = []
times = []
for i in 1:10
    (result, t) = @timed GAME(easy_noisy_sequences,0.001,3000, 500, 20)
    push!(times, t)
    push!(results, result)
    push!(scores, result[5])
end
results

print(results)

precision = precisionGetter(results, "CGCTCTCCAGCTCGTAGCTC", 20, easy_noisy_sequences)
println("Precision:\t", precision)

println(Statistics.mean(scores))
scores

Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[5, 6, 11, 41, 40, 11, 13, 39, 43, 42, 17, 20, 33, 42, 4, 11, 20, 36, 30, 38])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[7, 8, 13, 43, 2, 3, 15, 41, 45, 44, 19, 22, 35, 37, 6, 43, 22, 1, 32, 40])
└ @ Main In[14]:64


Aopt1


...

.0, 213.0, 215.0, 215.0, 215.0, 215.0, 215.0, 215.0, 215.0, 215.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 217.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 221.0, 222.0, 222.0, 222.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 223.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 224.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 226.0, 22

244.0, 244.0, 244.0, 244.0, 243.986, 244.0, 244.004, 244.008, 244.016, 244.02, 244.052, 244.102, 244.158, 244.35, 244.55, 244.772, 244.918, 244.99, 245.0, 245.0, 245.0, 245.0, 244.988, 245.0, 245.0, 244.988, 244.988, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 244.994, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.0, 245.004, 244.998, 245.006, 245.032, 245.062, 245.116, 245.218, 245.38, 245.588, 245.844, 245.97, 245.998, 246.0, 245.992, 246.0, 246.0, 246.0, 245.986, 246.0, 246.0, 246.0, 245.988, 246.0, 246.0, 246.0, 246.0, 245.994, 246.0, 246.0, 245.97, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 246.0, 245.988, 246.0, 246.0, 246.0, 246.0, 246.0, 245.996, 245.996], [48, 5, 10, 40, 29, 10, 12, 38, 42, 41, 16, 29, 32, 41, 3, 50, 19, 35, 29, 37], 251), (Any[167, 168, 170, 170, 170, 174, 176, 176, 179, 179, 180, 180, 180, 182, 182, 186, 189, 189, 192, 192, 194, 198, 198, 202, 207, 211, 211, 211, 211, 215, 215, 215, 218, 218, 222, 2

203.936, 205.274, 206.61, 207.862, 208.974, 209.956, 210.854, 211.692, 212.536, 213.344, 214.074, 214.646, 215.172, 215.73, 216.204, 216.712, 217.142, 217.568, 217.928, 218.222, 218.5, 218.81, 219.164, 219.56, 220.104, 220.762, 221.468, 222.2, 222.892, 223.522, 224.058, 224.524, 224.852, 225.09, 225.262, 225.508, 225.876, 226.44, 227.178, 228.16, 229.256, 230.11, 230.618, 230.928, 231.178, 231.45, 231.692, 231.896, 231.986, 232.008, 232.032, 232.054, 232.112, 232.23, 232.39, 232.652, 232.93, 233.158, 233.364, 233.616, 233.844, 233.98, 234.042, 234.128, 234.232, 234.42, 234.744, 235.188, 235.674, 236.04, 236.248, 236.516, 236.958, 237.444, 237.868, 238.002, 238.056, 238.096, 238.202, 238.366, 238.598, 238.842, 239.004, 239.106, 239.208, 239.358, 239.604, 239.894, 240.238, 240.526, 240.966, 241.346, 241.754, 242.114, 242.434, 242.724, 242.986, 243.174, 243.342, 243.566, 243.806, 243.964, 244.0, 244.008, 244.016, 244.028, 244.052, 244.102, 244.196, 244.342, 244.558, 244.808, 244.964, 244.

253.0, 253.0, 252.982, 253.0, 253.016, 253.032, 253.064, 253.128, 253.248, 253.464, 253.848, 254.528, 255.44, 256.368, 256.84, 257.0, 257.02, 257.04, 257.08, 257.14, 257.27, 257.54, 258.022, 258.81, 259.994, 261.338, 261.934, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 261.998, 261.97, 261.992, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 261.986, 262.0, 261.984, 262.0, 262.0, 261.992, 262.0, 262.0, 262.0, 262.0, 262.0, 261.986, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0, 262.0], [8, 9, 7, 44, 3, 4, 16, 42, 46, 45, 20, 23, 36, 38, 7, 3, 23, 2, 33, 41], 273)]Precision:	1.0
257.5


10-element Array{Any,1}:
 260
 272
 251
 236
 251
 266
 235
 262
 269
 273

In [37]:
testProfile = GenerateMotifStrings([8, 9, 7, 44, 3, 4, 16, 42, 46, 45, 20, 23, 36, 38, 7, 3, 23, 2, 33, 41], 20, easy_noisy_sequences)
println(testProfile)
testConsensus = ConsensusString(testProfile)
println(testConsensus)

Any["CGCTTTCATGCGATTAGCAC", "CGATCTCCAACTCGAATATC", "GGTTCTCCACTTTGCACCTC", "CGCTGTGTAGCTAATAGCAC", "TGGGCTTCCGCTCGAGGGTC", "CGATTCACGGCGCTTGCCTC", "GGGACACCAGTTCGTAGATG", "CGGTCTCGACGTCGTATCTG", "GGCCCCCCAGCACATAGCTT", "CGCTCCACAGCTCCTACCTC", "CGCTCTACAGCTCGTAGATC", "CGCCCTACAGGGCTAACCCC", "CGCTAACCAGCTCATAGCTC", "CGGTAGGCATCCCTCAGCTC", "CTCTCGACAGCCCGTTCATG", "AGATTTCCTGGCCGTCCCTT", "TGCTTTCCAACGCGTCGCTC", "TGCAGTGCCAGTCGTAACTC", "CACACCCAAGGTCTTAGCAC", "CGCTCTCCAGCACGTATGAC"]
CGCTCTCCAGCTCGTAGCTC


In [38]:
Statistics.mean(times)

11.821425183399999

Hard Challenge with 100% Motif Conservation

In [39]:
hard_sequences = []
open("Final Generated Fasta Files/Hard GGGCACCGAA-94,158,75,21,122,124,33,156,122,129,88,85,54,20,103,18,182,45,9,162,.fasta") do file
    i = 0
    for ln in eachline(file)
        i += 1 
        if i % 2 == 0
            push!(hard_sequences, strip(ln))
        end
    end
end
BAYESIAN_SCORING = false

results = []
scores = []
times = []
for i in 1:10
    (result, t) = @timed GAME(hard_sequences,0.001,3000, 500, 10)
    push!(times, t)
    push!(results, result)
    push!(scores, result[5])
end
results

print(results)

precision = precisionGetter(results, "GGGCACCGAA", 10, hard_sequences)
println("Precision:\t", precision)

println(Statistics.mean(scores))
scores

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[170, 101, 12, 105, 183, 108, 101, 113, 22, 142, 150, 27, 128, 99, 175, 64, 181, 37, 129, 35])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[89, 7, 92, 148, 2, 45, 195, 87, 116, 113, 124, 80, 162, 112, 70, 85, 15, 186, 74, 150])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[90, 9, 24, 24, 170, 103, 136, 11, 194, 68, 41, 174, 66, 172, 77, 124, 118, 108, 101, 132])
└ @ Main In[14]:64


...

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[55, 2, 1, 115, 181, 88, 25, 153, 185, 126, 85, 104, 156, 1, 8, 148, 13, 42, 183, 13])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[53, 161, 63, 24, 125, 135, 36, 159, 140, 10, 91, 88, 184, 23, 106, 21, 185, 48, 114, 165])
└ @ Main In[14]:64


Aopt1
Any[(Any[89, 90, 90, 93, 93, 93, 93, 93, 93, 94, 96, 96, 96, 96, 96, 97, 98, 98, 99, 101, 108, 108, 108, 108, 108, 108, 108, 109, 110, 110, 110, 111, 111, 112, 112, 114, 114, 114, 114, 114, 116, 116, 116, 117, 117, 118, 118, 118, 118, 118, 118, 119, 119, 119, 119, 119, 119, 120, 120, 120, 121, 121, 121, 121, 121, 122, 122, 123, 123, 123, 124, 124, 124, 124, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126], Any[76.0, 77.0, 79.0, 80.0, 81.0, 82.0, 83.0, 83.0, 84.0, 85.0, 86.0, 87.0, 87.0, 88.0, 89.0, 90.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 97.0, 98.0, 99.0, 100.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 108.0, 109.0, 109.0, 110.0, 110.0, 110.0, 111.0, 111.5, 112.0, 

10-element Array{Any,1}:
 139
 131
 138
 189
 139
 136
 200
 200
 141
 196

In [40]:
testProfile = GenerateMotifStrings([94, 158, 75, 21, 122, 124, 33, 156, 122, 129, 88, 85, 54, 20, 103, 18, 182, 45, 9, 162], 10, hard_sequences)
println(testProfile)
testConsensus = ConsensusString(testProfile)
println(testConsensus)

Any["GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA", "GGGCACCGAA"]
GGGCACCGAA


In [41]:
Statistics.mean(times)

5.437794461299999

Hard Challenge with 70% Motif Conservation

In [42]:
hard_noisy_sequences = []
open("Final Generated Fasta Files/Hard Noisy AGCATCAGTT-183,169,3,84,162,31,110,200,177,123,163,74,85,113,32,81,120,112,46,36,.fasta") do file
    i = 0
    for ln in eachline(file)
        i += 1 
        if i % 2 == 0
            push!(hard_noisy_sequences, strip(ln))
        end
    end
end
BAYESIAN_SCORING = false

results = []
scores = []
times = []
for i in 1:10
    (result, t) = @timed GAME(hard_noisy_sequences,0.001,3000, 500, 10)
    push!(times, t)
    push!(results, result)
    push!(scores, result[5])
end
results

print(results)

precision = precisionGetter(results, "AGCATCAGTT", 10, hard_noisy_sequences)
println("Precision:\t", precision)

println(Statistics.mean(scores))
scores

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[179, 199, 127, 182, 116, 58, 71, 149, 179, 124, 59, 76, 186, 4, 98, 144, 125, 97, 101, 38])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[35, 87, 133, 46, 189, 170, 154, 173, 81, 63, 21, 16, 162, 124, 94, 1, 159, 188, 196, 72])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[173, 99, 154, 11, 145, 5, 200, 16, 130, 162, 119, 161, 136, 59, 196, 200, 6, 87, 109, 86])
└ @ Main In[14]:64


...

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[104, 107, 28, 197, 152, 73, 147, 14, 178, 161, 66, 154, 51, 52, 162, 199, 118, 151, 108, 37])
└ @ Main In[14]:64


Aopt1


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[75, 170, 136, 107, 38, 84, 169, 113, 187, 4, 65, 72, 172, 33, 119, 104, 193, 124, 172, 147])
└ @ Main In[14]:64


Aopt1
Any[(Any[86, 88, 88, 89, 91, 91, 92, 92, 93, 93, 94, 95, 95, 98, 98, 99, 99, 99, 99, 100, 100, 103, 103, 103, 103, 103, 104, 104, 105, 105, 105, 106, 107, 107, 107, 107, 108, 108, 108, 108, 108, 108, 111, 111, 111, 111, 111, 112, 112, 112, 112, 113, 114, 115, 115, 115, 115, 115, 116, 116, 116, 116, 116, 117, 117, 117, 117, 117, 117, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 123, 123, 123, 123, 123, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 128, 128, 128, 128, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129,

10-element Array{Any,1}:
 140
 136
 136
 137
 137
 138
 135
 127
 138
 133

In [43]:
testProfile = GenerateMotifStrings([39, 79, 38, 48, 104, 88, 31, 146, 18, 38, 187, 195, 133, 184, 80, 83, 119, 157, 179, 24], 10, hard_noisy_sequences)
println(testProfile)
testConsensus = ConsensusString(testProfile)
println(testConsensus)

Any["GCGCACCCGA", "CCGCACTATA", "CCTGACCAGG", "ACTCACCAGG", "TCGCCCTCGA", "CCACACTATA", "TCGCAACAGA", "CCGAACTAGT", "GCCCAACTGA", "CCGCCTCACG", "ACGTGGCAGA", "CTGGACCAGC", "GAGCATCAGA", "ACGTACCAGA", "CCGGCCCAGC", "CATGACCTGA", "CCGCAGCAGT", "CCTTACTAGA", "CGGCACGTGA", "CCGAACCATT"]
CCGCACCAGA


In [44]:
Statistics.mean(times)

5.557933976399999

Tests for Graphs

1) Easy Challenge Mutation vs Score

In [45]:
println(easy_sequences)
step = 0.05
mutrate_score = []
r = 0.001
while r < 1
    result = @time GAME(easy_sequences,r,3000, 500, 20)
    push!(mutrate_score, (r, result[5]))
    r += step
end
mutrate_score

Any["GTGCTCCGGCAGTGGTGCTTAGTATCATCGTGAGGGATAAAAATAAAAGCGGATGTGCAACCCAGTAGGC", "CCTACAAGCTTTTTGTGGACCCAGTGATCGTTATTGACAGGATATCGCTGGATAAAAATAAAAGCGGATA", "TGTCTGCTGCACGGCAGGATAAAAATAAAAGCGGATTCCTTGTATATCTCGTGTAAATACGGACTGGGAG", "TGGATAAAAATAAAAGCGGATTTTGCCGCTTTGAGCCTCGTTACTGTGCGAGTAGATTCAACCCATTATC", "TCCGGTTGCACTTGCAAAATCTCGGCGCGAATTGAGTATAGGATAAAAATAAAAGCGGATCCTCACCCGC", "CACCCTTGGATTCAAAGAGAGGCTTGGGAAATTATATTTGCATCAGGATAAAAATAAAAGCGGATCTCGT", "GTAAGAACCATAATTTACCACGTCATTTTGCATGAGACACCATTAGTGGGATAAAAATAAAAGCGGATGT", "ATAAGCCCGGTTTTAAGTCGCCGAGGATAAAAATAAAAGCGGATTAGCCCAGCAAACTTTTCGAACTATG", "CACGCGCCCGCGGGGATAAAAATAAAAGCGGATATCAGTGTTCAATTGGCTTACGCTGTAGGCATAGACG", "CCGTGGGATAAAAATAAAAGCGGATGATATGTAACTAAACAAACGCACGTCGTGATCTCTCGTTGTGCAT", "TTAGCTGTAGGAAAATAGCACGGATAAAAATAAAAGCGGATTTAGTACTAAACTCCGGCCTCATTATGTA", "GCTGTGACGGGACGATACGCATCCAAAGGAACGTCGGTGGATAAAAATAAAAGCGGATAAAGGAAATTGC", "TCCCGGATAAAAATAAAAGCGGATAGCGTGCCCCAGAGGAGCGTGTCTGGGCCGTTGGCCTCATATGAGC", "GTGCATTAGGATAAAAATAAAAGCGGATGAAAA

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[34, 49, 16, 1, 40, 45, 48, 24, 13, 5, 21, 38, 4, 8, 37, 7, 5, 16, 7, 4])
└ @ Main In[14]:64


Aopt457
 12.663863 seconds (90.86 M allocations: 8.261 GiB, 9.43% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[34, 49, 16, 1, 40, 45, 48, 24, 13, 5, 21, 38, 4, 8, 37, 7, 5, 16, 7, 4])
└ @ Main In[14]:64


Aopt154
137.330742 seconds (929.79 M allocations: 84.525 GiB, 8.93% gc time)


...

Aopt5
144.985833 seconds (929.98 M allocations: 84.544 GiB, 8.61% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[33, 9, 18, 26, 37, 44, 46, 18, 30, 3, 20, 15, 9, 11, 37, 10, 17, 41, 18, 9])
└ @ Main In[14]:64


Aopt377
145.104856 seconds (929.79 M allocations: 84.525 GiB, 8.62% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[1, 32, 51, 1, 39, 44, 3, 30, 12, 29, 4, 44, 6, 45, 1, 7, 33, 19, 8, 24])
└ @ Main In[14]:64


20-element Array{Any,1}:
 (0.001, 400)               
 (0.051000000000000004, 400)
 (0.101, 400)               
 (0.15100000000000002, 400) 
 (0.201, 400)               
 (0.251, 400)               
 (0.301, 375)               
 (0.351, 400)               
 (0.40099999999999997, 400) 
 (0.45099999999999996, 400) 
 (0.501, 400)               
 (0.551, 400)               
 (0.6010000000000001, 400)  
 (0.6510000000000001, 400)  
 (0.7010000000000002, 374)  
 (0.7510000000000002, 400)  
 (0.8010000000000003, 400)  
 (0.8510000000000003, 388)  
 (0.9010000000000004, 390)  
 (0.9510000000000004, 390)  

In [5]:
2) Hard Challenge Mutation vs Score

LoadError: syntax: extra token ")" after end of expression

In [53]:
step = 0.025
mutrate_score_hard = []
r = 0.001
while r < 1
    result = @time GAME(hard_sequences,r,3000, 500, 10)
    push!(mutrate_score_hard, (r, result[5]))
    r += step
end
mutrate_score_hard

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[93, 158, 133, 35, 199, 106, 187, 155, 121, 46, 94, 85, 17, 175, 155, 78, 163, 32, 154, 84])
└ @ Main In[14]:64


Aopt1
  8.156892 seconds (63.54 M allocations: 5.136 GiB, 10.41% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[191, 190, 185, 23, 141, 134, 128, 83, 28, 57, 90, 87, 196, 152, 53, 104, 68, 105, 19, 145])
└ @ Main In[14]:64


Aopt1
  5.419540 seconds (40.94 M allocations: 3.309 GiB, 10.09% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[99, 145, 115, 116, 162, 1, 38, 39, 29, 19, 49, 30, 66, 35, 112, 56, 18, 114, 101, 91])
└ @ Main In[14]:64


...

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[119, 200, 80, 109, 106, 67, 195, 104, 154, 196, 185, 16, 48, 94, 45, 48, 152, 198, 22, 152])
└ @ Main In[14]:64


Aopt43
 82.414998 seconds (615.87 M allocations: 49.744 GiB, 8.35% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[125, 10, 87, 136, 125, 76, 72, 156, 11, 109, 53, 63, 154, 198, 117, 186, 28, 24, 1, 168])
└ @ Main In[14]:64


Aopt254
 82.648130 seconds (616.85 M allocations: 49.826 GiB, 8.37% gc time)


40-element Array{Any,1}:
 (0.001, 200)               
 (0.026000000000000002, 138)
 (0.051000000000000004, 140)
 (0.07600000000000001, 198) 
 (0.101, 200)               
 (0.126, 200)               
 (0.151, 139)               
 (0.176, 200)               
 (0.20099999999999998, 135) 
 (0.22599999999999998, 139) 
 (0.251, 133)               
 (0.276, 139)               
 (0.30100000000000005, 135) 
 ⋮                          
 (0.7010000000000004, 200)  
 (0.7260000000000004, 200)  
 (0.7510000000000004, 133)  
 (0.7760000000000005, 133)  
 (0.8010000000000005, 130)  
 (0.8260000000000005, 141)  
 (0.8510000000000005, 135)  
 (0.8760000000000006, 137)  
 (0.9010000000000006, 139)  
 (0.9260000000000006, 136)  
 (0.9510000000000006, 134)  
 (0.9760000000000006, 200)  

3) Easy Challenge Population vs Score

In [47]:
step = 1
pop_score = []
p = 2
while p <= 500
    result = @time GAME(easy_sequences,0.001,3000, p, 20)
    push!(pop_score, (p, result[5]))
    p += step
end
pop_score

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[45, 35, 17, 34, 36, 48, 18, 32, 24, 6, 1, 43, 33, 32, 18, 51, 50, 17, 16, 1])
└ @ Main In[14]:64


Aopt1
  0.070836 seconds (467.38 k allocations: 44.499 MiB, 12.99% gc time)
Aopt1
  0.093705 seconds (664.63 k allocations: 63.527 MiB, 5.51% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[1, 44, 47, 39, 44, 34, 23, 22, 5, 13, 25, 42, 28, 13, 51, 7, 29, 15, 24, 2])
└ @ Main In[14]:64


Aopt1
  0.106414 seconds (725.60 k allocations: 69.194 MiB, 9.54% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[31, 40, 18, 8, 6, 51, 11, 31, 50, 22, 26, 51, 31, 8, 25, 13, 22, 13, 12, 21])
└ @ Main In[14]:64


...

Aopt1
  8.897940 seconds (64.90 M allocations: 5.901 GiB, 10.49% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[34, 49, 16, 2, 40, 45, 48, 24, 13, 5, 21, 12, 4, 8, 37, 7, 5, 16, 7, 4])
└ @ Main In[14]:64


Aopt1
  5.164244 seconds (36.97 M allocations: 3.361 GiB, 10.33% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[35, 50, 17, 3, 41, 46, 49, 25, 14, 6, 22, 39, 7, 9, 38, 8, 6, 17, 8, 5])
└ @ Main In[14]:64


499-element Array{Any,1}:
 (2, 400)  
 (3, 374)  
 (4, 375)  
 (5, 346)  
 (6, 400)  
 (7, 368)  
 (8, 388)  
 (9, 368)  
 (10, 390) 
 (11, 388) 
 (12, 375) 
 (13, 400) 
 (14, 388) 
 ⋮         
 (489, 400)
 (490, 400)
 (491, 400)
 (492, 400)
 (493, 400)
 (494, 400)
 (495, 400)
 (496, 400)
 (497, 400)
 (498, 400)
 (499, 400)
 (500, 400)

4) Hard Challenge Mutation vs Score

In [55]:
step = 1
pop_score_hard = []
p = 2
while p <= 500
    result = @time GAME(hard_sequences,0.001,3000, p, 10)
    push!(pop_score_hard, (p, result[5]))
    p += step
end
pop_score_hard

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[196, 141, 41, 140, 173, 106, 105, 158, 139, 73, 40, 200, 11, 25, 141, 141, 28, 142, 5, 26])
└ @ Main In[14]:64


Aopt1
  0.248346 seconds (2.02 M allocations: 173.779 MiB, 9.15% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[40, 90, 117, 132, 177, 158, 130, 30, 96, 58, 98, 81, 145, 8, 111, 58, 170, 141, 103, 170])
└ @ Main In[14]:64


Aopt1
  0.205329 seconds (1.59 M allocations: 136.259 MiB, 7.74% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[128, 168, 48, 193, 74, 45, 151, 53, 12, 110, 1, 136, 126, 93, 36, 90, 124, 148, 152, 68])
└ @ Main In[14]:64


...

┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[37, 59, 86, 18, 50, 121, 30, 153, 119, 126, 99, 104, 69, 17, 194, 98, 179, 43, 6, 159])
└ @ Main In[14]:64


Aopt1
  9.081796 seconds (70.77 M allocations: 5.718 GiB, 10.47% gc time)


┌ Info: Genetic screen finished...
└ @ Main In[14]:63
┌ Info: ("Aopt_motif", Any[151, 54, 64, 164, 96, 115, 24, 72, 19, 72, 59, 24, 34, 59, 63, 61, 159, 174, 118, 69])
└ @ Main In[14]:64


Aopt1
  7.758823 seconds (58.15 M allocations: 4.699 GiB, 10.08% gc time)


499-element Array{Any,1}:
 (2, 200)  
 (3, 132)  
 (4, 141)  
 (5, 131)  
 (6, 136)  
 (7, 135)  
 (8, 134)  
 (9, 138)  
 (10, 134) 
 (11, 133) 
 (12, 133) 
 (13, 200) 
 (14, 133) 
 ⋮         
 (489, 135)
 (490, 200)
 (491, 140)
 (492, 136)
 (493, 200)
 (494, 132)
 (495, 142)
 (496, 200)
 (497, 200)
 (498, 135)
 (499, 184)
 (500, 131)

In [49]:
open("easy_population_v_score.data", "w") do file
        println(file, pop_score)
end

In [50]:
open("hard_population_v_score.data", "w") do file
        println(file, pop_score_hard)
end

In [54]:
open("hard_mutation_v_score_twice.data", "w") do file
        println(file, mutrate_score_hard)
end

In [52]:
open("easy_mutation_v_score.data", "w") do file
        println(file, mutrate_score)
end

In [6]:
# The end.